### **Intro Tutorial**

This Notebook will show the basic principle of using hera. And how to submit Workflows which will run on a schedule.

**First some imports**

In [ ]:
from hera.workflows import  script, DAG, Workflow, CronWorkflow
from hera.shared import global_config

**Global settings**

To commit workflows to created with hera to ArgoWorkflows, we need to set some global parameters. `host`, `token` and `namespace` provide hera the information on where to submit the workflow. A globally set image will be used for all of of the steps in the workflow. It is possible to also define a seperate image for each step.

In [ ]:
global_config.host = "https://dev.services.eodc.eu/workflows/"
global_config.namespace = "<YOUR NAMESPACE>"
global_config.token = "<YOUR TOKEN>"
global_config.image = "ghcr.io/eodcgmbh/cluster_image:2025.2.0"

**Writing scripts**

With the `@script` decorator we can define the steps of our workflow. We can set different parameters in the decorator, like a seperate image, but we will go into detail in later notebooks. Each step in our workflow needs to be in a function and have its own script decorator.

For now we will define a simple function which prints our input.

In [ ]:
@script()
def hello_hera(message):
    print(message)

**Creating the Workflow**

With `Workflow` we can create the workflow. The Workflow needs to have `name` or `generate` name set, also `entrypoint` has to define where the workflow will start, if we don't define `namespace` or `image` globally, we can also do it here. With `DAG` we can tell hera the steps of the workflow, if the same function is used more than once in a workflow it will need a specific name for each time it is used. The parameters in the function are passed with the `arguments` parameter as  a dictionary. Lastly, the flow of the workflow is set with the `>>` operator, whereas, steps in squared brackets will run parallel.

In [ ]:
with Workflow(
    generate_name = "hello-hera-",
    entrypoint = "first-steps"
) as w:
    with DAG(name="first-steps"):
        first = hello_hera(name="hello", arguments={"message": "Hello hera!"})
        
        second_1 = hello_hera(name="parallel1", arguments={"message": "This message gets printed in parallel."})
        second_2 = hello_hera(name="parallel2", arguments={"message": "This message gets printed in parallel."})

        last = hello_hera(name="goodbye", arguments={"message": "Goodbye hera!"})

        first >> [second_1, second_2] >> last

**Submitting the Workflow**

Lastly the workflow can be submited to Argo with `.create()`

In [ ]:
w.create()

**CronWorkflow**

We can also create a `Cronworkflow` which will run at a given schedule. Note that it will run in UTC time.

In [ ]:
with CronWorkflow(
    generate_name="cron-workflow-",
    schedule="* 10 * * *",
    entrypoint="workflow"
) as w:
    with DAG(name="workflow"):
        hello_hera({"message": "Hello hera!"})